# Project 3 - Part 2 Extract from TMDB (Core)

## Import Libraries | Download Data 

In [1]:
import os, time,json
import pandas as pd
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

## Custom Functions

In [2]:
## Custom Functions

def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    info = movie.info()

    # save the .info .releases dictionaries
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        #if the coutry abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## save a 'certification' key in info with the certification
            info['certification'] = c['certification']
    return info



In [3]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## API Keys | Call API

In [4]:
with open('/Users/dmburns/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [5]:
tmdb.API_KEY =  login['api-key']

## Test Function

In [7]:
## testing our function by looping through a list of ids

test_ids = ["tt0332280", "tt0848228"]
results = []
for movie_id in test_ids:
    movie_info = get_movie_with_rating(movie_id)
    results.append(movie_info)
    
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,10024,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.708,27977,PG-13


## Prepare for loops

In [10]:
#BEFORE THE LOOPS
#Designate a folder
#You will save API call data in the data folder you created for project part 1

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['akas_clean.csv.gz',
 '.DS_Store',
 'basics.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'ratings_clean.csv.gz',
 'tmdb_api_results_2001.json',
 'ratings.csv.gz',
 'akas.csv',
 'final_tmdb_data_2000.csv',
 'akas.csv.gz',
 'basics_clean.csv.gz',
 'final_tmdb_data_2001.csv.gz']

In [11]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/basics_clean.csv.gz')

In [12]:
# create required lists for loop

YEARS_TO_GET = [2000,2001]

In [13]:
# define an errors list

errors = [ ]

## Set up loops

In [14]:
# Outer Loop
# set up progress bar
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy() 

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]    
    
    # Start Inner Loop

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        except Exception as e:
            errors.append([movie_id, e])

            final_year_df = pd.read_json(JSON_FILE)
            final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)    

print(f"- Total errors: {len(errors)}")    

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1421 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1532 [00:00<?, ?it/s]

- Total errors: 446
